# ML Options Pricing Model

### This notebook replicates the functionality of `main.py` while adding visualizations, statistical comparisons, and regression analysis.

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model
from scipy.interpolate import griddata

In [ ]:
from data_loader import get_options_chain, parse_options_chain
from volatility_model import VolatilityModel
from pricing_model import PricingModel
from signal_generator import SignalGenerator

In [ ]:
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

### Calculate historical volatility using GARCH

In [ ]:
ticker = 'AAPL'

In [ ]:
options_json = get_options_chain(ticker)
options_json

In [ ]:
options = parse_options_chain(options_json)

In [ ]:
stock_data = yf.download(ticker, period="5y")

In [ ]:
# get the interest rate from an api 10 year treasury yield
interest_rate = yf.Ticker("^TNX").history(period="1d")["Close"].iloc[-1]

In [ ]:
print("Options Chain:")
print(options.head())

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(stock_data['Close'], label='Close Price')
plt.title(f"{ticker} Historical Stock Price")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()

In [ ]:
stock_data['LogReturn'] = np.log(stock_data['Close'] / stock_data['Close'].shift(1))
stock_data.dropna(inplace=True)

In [ ]:
stock_data['LogReturn'] *= 100

In [ ]:
garch_model = arch_model(stock_data['LogReturn'], vol='Garch', p=1, q=1)
garch_model_fit = garch_model.fit(disp='off')

In [ ]:
print(garch_model_fit.summary())

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(garch_model_fit.conditional_volatility, label='Conditional Volatility')
plt.title(f"{ticker} Conditional Volatility (GARCH)")
plt.xlabel("Time")
plt.ylabel("Volatility")
plt.legend()
plt.show()

In [ ]:
underlying_price = stock_data['Close'].iloc[-1]
risk_free_rate = interest_rate / 100

In [ ]:
volModel = VolatilityModel(options_chain_df=options, underlying_price=underlying_price, risk_free_rate=risk_free_rate)
historical_volatility = garch_model_fit.conditional_volatility

In [ ]:
options['ImpliedVolatility'] = options.apply(
    lambda row: volModel.calculate_implied_volatility(
        option_price=row['mark'],
        strike=row['strikePrice'],
        expiration_days=row['daysToExpiration'],
        option_type=row['putCall'],  # 'CALL' or 'PUT'
    ),
    axis=1
)

In [ ]:
# Display the first few rows of the options chain with implied volatilities
print("Options Chain with Implied Volatilities:")
print(options.head())

In [ ]:
# Plot implied volatility vs strike price
plt.figure(figsize=(10, 6))
plt.scatter(options['strikePrice'], options['ImpliedVolatility'], label='Implied Volatility')
plt.title(f"{ticker} Implied Volatility vs Strike Price")
plt.xlabel("Strike Price")
plt.ylabel("Implied Volatility")
plt.legend()
plt.show()

In [ ]:
strikes = options['strikePrice'].values
expirations = options['daysToExpiration'].values
implied_vols = options['ImpliedVolatility'].values

In [ ]:
grid_strikes, grid_expirations = np.meshgrid(
    np.linspace(strikes.min(), strikes.max(), 100),
    np.linspace(expirations.min(), expirations.max(), 100)
)

In [ ]:
# Add small noise to strikes and expirations
noise_level = 1e-5  # Adjust this value as needed
strikes = options['strikePrice'].values + np.random.normal(0, noise_level, size=len(options))
expirations = options['daysToExpiration'].values + np.random.normal(0, noise_level, size=len(options))
implied_vols = options['ImpliedVolatility'].values

# Create a grid for interpolation
grid_strikes, grid_expirations = np.meshgrid(
    np.linspace(strikes.min(), strikes.max(), 100),
    np.linspace(expirations.min(), expirations.max(), 100)
)

# Interpolate using cubic spline
volatility_surface = griddata(
    (strikes, expirations), implied_vols,
    (grid_strikes, grid_expirations), method='cubic'
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.contourf(grid_strikes, grid_expirations, volatility_surface, levels=50, cmap='viridis')
plt.colorbar(label='Implied Volatility')
plt.title(f"{ticker} Volatility Surface")
plt.xlabel("Strike Price")
plt.ylabel("Days to Expiration")
plt.show()

In [ ]:
underlying_price = stock_data['Close'].iloc[-1]
options['Moneyness'] = underlying_price[0] / options['strikePrice']
options['TimeToExpiration'] = options['daysToExpiration'] / 365
options['RiskFreeRate'] = interest_rate / 100
options['HistoricalVolatility'] = garch_model_fit.conditional_volatility[-1] / 100

In [ ]:
print("Options Chain with Features:")
print(options.head())

### Train the XGBoost model

In [ ]:
pricing_model = PricingModel()
pricing_model.train_xgboost_model(options, underlying_price[0], interest_rate/100, historical_volatility[0])

In [ ]:
predicted_chain = pricing_model.predict_option_chain(
    stock_price=stock_data['Close'].iloc[-1],
    options_chain=options,
    interest_rate=interest_rate,
    historical_volatility=historical_volatility.iloc[-1]
)

In [ ]:
print("Predicted Option Chain:")
print(predicted_chain.head())

### Trading Signals

In [ ]:
signal_generator = SignalGenerator()
signals = signal_generator.generate_trading_signals(predicted_chain)

In [ ]:
print("\nTrading Signals:")
print(signal_generator.format_output(signals))

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(signals['priceDifference'], bins=30, kde=True, color="purple")
plt.title("Distribution of Price Differences (Predicted vs. Market)")
plt.xlabel("Price Difference")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(x="strikePrice", y="priceDifference", hue="signal", data=signals, palette="viridis")
plt.title("Trading Signals by Strike Price and Price Difference")
plt.xlabel("Strike Price")
plt.ylabel("Price Difference")
plt.legend(title="Signal")
plt.show()